# DuckDB + MovieLens 1M — Starter Notebook (VS Code)

In [1]:
#%pip install -q duckdb python-dotenv

import os, duckdb
from dotenv import load_dotenv
load_dotenv()  # lê .env com as credenciais do Learner Lab

con = duckdb.connect("movielens1m.duckdb")  # opcional: ficheiro local
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.execute(f"SET s3_region='{os.getenv('AWS_DEFAULT_REGION','us-east-1')}';")
con.execute(f"SET s3_access_key_id='{os.getenv('AWS_ACCESS_KEY_ID')}';")
con.execute(f"SET s3_secret_access_key='{os.getenv('AWS_SECRET_ACCESS_KEY')}';")
con.execute(f"SET s3_session_token='{os.getenv('AWS_SESSION_TOKEN')}';")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Views que apontam para os Parquet no S3 (mesmos nomes que no 100k)

In [2]:
BASE = "s3://bdf25-20-movielens/curated"

con.execute(f"CREATE OR REPLACE VIEW movies  AS SELECT * FROM read_parquet('{BASE}/movies/*.parquet');")
con.execute(f"CREATE OR REPLACE VIEW ratings AS SELECT * FROM read_parquet('{BASE}/ratings/*.parquet');")
con.execute(f"CREATE OR REPLACE VIEW users   AS SELECT * FROM read_parquet('{BASE}/users/*.parquet');")

# opcional: verificação
con.execute("SHOW TABLES").df()
con.execute("SELECT * FROM movies LIMIT 5").df()


IOException: IO Error: No files found that match the pattern "s3://bdf25-20-movielens/curated/movies/*.parquet"

LINE 1: CREATE OR REPLACE VIEW movies  AS SELECT * FROM read_parquet('s3://bdf25-20-movielens/curated/movies/*.parquet...
                                                        ^

In [ ]:
# cria ou liga se já existir:
con = duckdb.connect("movielens1m.duckdb")

# ver tabelas existentes
existing = set(con.sql("SHOW TABLES").df()["name"].str.lower())

# criar tabelas só se ainda não existirem
# movies (importação directa sem alterações — é texto e ok)
if "movies" not in existing:
    con.sql(f"CREATE TABLE movies AS SELECT * FROM read_csv_auto('{movies_path}')")

# ratings (na importação é importante garantir timestamp correcto)
con.sql(f"""
CREATE OR REPLACE TABLE ratings AS
SELECT
    userId::INT     As userId,
    movieId::INT    As movieId,
    rating::DOUBLE  As rating,
    to_timestamp(CAST(timestamp AS BIGINT)) AS timestamp
FROM read_csv_auto('{ratings_path}')
""")

# tags (também garantir timestamp)
con.sql(f"""
CREATE OR REPLACE TABLE tags AS
SELECT
    userId::INT     As userId,
    movieId::INT    As movieId,
    tag,
    to_timestamp(CAST(timestamp AS BIGINT)) AS timestamp
FROM read_csv_auto('{tags_path}')
""")

# links (tudo texto está ok)
if "links" not in existing:
    con.sql(f"CREATE TABLE links AS SELECT * FROM read_csv_auto('{links_path}')")

# mostrar estado final
con.sql("SHOW TABLES").df()

#Isto é melhor porquê?
#userId e movieId ficam logo como inteiros (não como texto)
#rating fica como número decimal real
#o timestamp passa a tipo TIMESTAMP (ts)


IOException: IO Error: No files found that match the pattern "..\data\100k\movies.csv"

LINE 1: CREATE TABLE movies AS SELECT * FROM read_csv_auto('..\data\100k\movies.csv')
                                             ^

In [ ]:
con.close()
print("Ligação fechada.")


Ligação fechada.
